In [2]:
import os
import pyspark
from pyspark.sql import SparkSession

memory = '2g'
pyspark_submit_args = ' --driver-memory ' + memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/26 16:28:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/26 16:28:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/02/26 16:28:22 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-02-25 21:59:39--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.216.107.188
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.216.107.188|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699.83M  5.85MB/s    in 4m 20s  

2022-02-25 22:03:59 (2.69 MB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [5]:
# Question 1: Version of PySpark *
spark.version

'3.2.1'

In [6]:
from pyspark.sql import types
schema = types.StructType([
	types.StructField('hvfhs_license_num',types.StringType(),True),
	types.StructField('dispatching_base_num',types.StringType(),True),
	types.StructField('pickup_datetime',types.TimestampType(),True),
	types.StructField('dropoff_datetime',types.TimestampType(),True),
	types.StructField('PULocationID',types.IntegerType(),True),
	types.StructField('DOLocationID',types.IntegerType(),True),
	types.StructField('SR_Flag',types.StringType(),True)
])

df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|          35|          39|   null|
|           HV0003|              B02764|2021-02-01 00:27:23|2021-02-01 00:44:01|          39|          35|   null|
|           HV0005|              B02510|2021-02-01 00:28:38|2021-02-01 00:38:27|          39|          91|   null|
|           HV0005|              B02510|2021-02-01 00:43:37|2021-02-01 01:23:20|          91|         228|   null|
|           HV0003|              B02872|2021-02-01 00:08:42|2021-02-01 00:17:57|         126|         250|   null|
|           HV0003|              B02872|2021-02-01 00:26:02|2021-02-01 00:42:51|

In [7]:
# This is important to update the actual dataframe
df = df.repartition(24)
df.write.parquet('fhvhv/2021/02/')

22/02/26 16:34:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 96.51% for 14 writers
22/02/26 16:34:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 90.08% for 15 writers
22/02/26 16:34:29 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 84.45% for 16 writers
22/02/26 16:34:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 90.08% for 15 writers
22/02/26 16:34:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 96.51% for 14 writers


In [13]:
!ls -lrth fhvhv/2021/02

total 442432
-rw-r--r--  1 sengopal  staff   8.4M Feb 26 16:34 part-00007-96c9c86d-9dbb-499d-a19d-b63d0dff98c1-c000.snappy.parquet
-rw-r--r--  1 sengopal  staff   8.4M Feb 26 16:34 part-00002-96c9c86d-9dbb-499d-a19d-b63d0dff98c1-c000.snappy.parquet
-rw-r--r--  1 sengopal  staff   8.4M Feb 26 16:34 part-00012-96c9c86d-9dbb-499d-a19d-b63d0dff98c1-c000.snappy.parquet
-rw-r--r--  1 sengopal  staff   8.4M Feb 26 16:34 part-00009-96c9c86d-9dbb-499d-a19d-b63d0dff98c1-c000.snappy.parquet
-rw-r--r--  1 sengopal  staff   8.4M Feb 26 16:34 part-00006-96c9c86d-9dbb-499d-a19d-b63d0dff98c1-c000.snappy.parquet
-rw-r--r--  1 sengopal  staff   8.4M Feb 26 16:34 part-00004-96c9c86d-9dbb-499d-a19d-b63d0dff98c1-c000.snappy.parquet
-rw-r--r--  1 sengopal  staff   8.4M Feb 26 16:34 part-00014-96c9c86d-9dbb-499d-a19d-b63d0dff98c1-c000.snappy.parquet
-rw-r--r--  1 sengopal  staff   8.4M Feb 26 16:34 part-00013-96c9c86d-9dbb-499d-a19d-b63d0dff98c1-c000.snappy.parquet
-rw-r--r--  1 sengopal  staff   8.4M Feb 26

In [ ]:
# Question 2: Size of HVFHW February 2021 *
# Repartition it to 24 partitions and save it to parquet.
# What's the size of the folder with results (in MB)?


In [14]:
df.count()

11613942

In [25]:
fhv_2021_02 = df.registerTempTable('fhv_2021_02')

In [26]:
# Q3: How many taxi trips were there on February 15?
# Consider only trips that started on February 15.

df_trips_feb15 = spark.sql("""
SELECT 
    date_trunc('date', pickup_datetime) AS date, 
    COUNT(1) AS number_records
FROM
    fhv_2021_02
WHERE   
    date_format(pickup_datetime,'yyyy-MM-dd') = '2021-02-15'
GROUP BY
    1
""")

In [27]:
df_trips_feb15.show()

+----+--------------+
|date|number_records|
+----+--------------+
|null|        367170|
+----+--------------+



In [24]:
## Question 4. Longest trip for each day. Now calculate the duration for each trip. 
## Trip starting on which day was the longest?

In [33]:
from pyspark.sql.functions import *
df_diff = df.withColumn('DiffInSeconds',col('dropoff_datetime').cast("long") - col('pickup_datetime').cast("long"))

In [34]:
df_diff.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,TimestampType,true),StructField(dropoff_datetime,TimestampType,true),StructField(PULocationID,IntegerType,true),StructField(DOLocationID,IntegerType,true),StructField(SR_Flag,StringType,true),StructField(DiffInSeconds,LongType,true)))

In [32]:
fhv_2021_02_duration = df_diff.registerTempTable('fhv_2021_02_duration')

In [42]:
df_trips_longest = spark.sql("""
SELECT 
    pickup_datetime, MAX(DiffInSeconds)
FROM
    (SELECT *, MAX(DiffInSeconds) OVER (PARTITION BY pickup_datetime) AS maxDiffInSeconds FROM fhv_2021_02_duration) M WHERE DiffInSeconds = maxDiffInSeconds
""")

df_trips_longest.show()

# "SELECT A, B FROM (SELECT *, MAX(B) OVER (PARTITION BY A) AS maxB FROM table) M WHERE B = maxB"


+-------------------+-------------+
|    pickup_datetime|DiffInSeconds|
+-------------------+-------------+
|2021-02-01 00:00:16|          819|
|2021-02-01 00:01:07|          918|
|2021-02-01 00:01:11|         1201|
|2021-02-01 00:01:17|         1040|
|2021-02-01 00:01:36|          564|
|2021-02-01 00:02:09|          608|
|2021-02-01 00:02:20|         1604|
|2021-02-01 00:02:31|          698|
|2021-02-01 00:03:14|          724|
|2021-02-01 00:03:45|          971|
|2021-02-01 00:03:58|         1328|
|2021-02-01 00:04:05|         1180|
|2021-02-01 00:04:25|          507|
|2021-02-01 00:04:49|         1139|
|2021-02-01 00:05:00|          705|
|2021-02-01 00:05:04|         1467|
|2021-02-01 00:05:34|         1019|
|2021-02-01 00:05:53|         1864|
|2021-02-01 00:06:09|         1436|
|2021-02-01 00:07:08|         1144|
+-------------------+-------------+
only showing top 20 rows



In [45]:
df_trips_longest = spark.sql("""
SELECT 
    pickup_datetime, DiffInSeconds
FROM
    fhv_2021_02_duration ORDER BY DiffInSeconds DESC
""")
df_trips_longest.head(5)

[Row(pickup_datetime=datetime.datetime(2021, 2, 11, 13, 40, 44), DiffInSeconds=75540),
 Row(pickup_datetime=datetime.datetime(2021, 2, 17, 15, 54, 53), DiffInSeconds=57221),
 Row(pickup_datetime=datetime.datetime(2021, 2, 20, 12, 8, 15), DiffInSeconds=44039),
 Row(pickup_datetime=datetime.datetime(2021, 2, 3, 20, 24, 25), DiffInSeconds=40653),
 Row(pickup_datetime=datetime.datetime(2021, 2, 19, 23, 17, 44), DiffInSeconds=37577)]

In [46]:
df_collect = df_diff.agg(max('DiffInSeconds').alias('B')).collect()


AttributeError: 'list' object has no attribute 'show'

In [47]:
df_collect

[Row(B=75540)]

In [ ]:
# Question 5. Most frequent dispatching_base_num
# Now find the most frequently occurring dispatching_base_num in this dataset.
# How many stages this spark job has?
# Note: the answer may depend on how you write the query, so there are multiple correct answers. Select the one you have.



In [48]:
df_trips_dispatch_num = spark.sql("""
SELECT 
    dispatching_base_num,
    COUNT(1) AS number_records
FROM
    fhv_2021_02
GROUP BY
    1
ORDER BY number_records DESC  
""")
df_trips_dispatch_num.head(5)

[Row(dispatching_base_num='B02510', number_records=3233664),
 Row(dispatching_base_num='B02764', number_records=965568),
 Row(dispatching_base_num='B02872', number_records=882689),
 Row(dispatching_base_num='B02875', number_records=685390),
 Row(dispatching_base_num='B02765', number_records=559768)]

In [49]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 2, 3, 11, 15, 45), dropoff_datetime=datetime.datetime(2021, 2, 3, 11, 24, 35), PULocationID=75, DOLocationID=263, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02865', pickup_datetime=datetime.datetime(2021, 2, 1, 5, 49, 35), dropoff_datetime=datetime.datetime(2021, 2, 1, 6, 9, 19), PULocationID=62, DOLocationID=65, SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime=datetime.datetime(2021, 2, 2, 8, 20, 19), dropoff_datetime=datetime.datetime(2021, 2, 2, 8, 30, 8), PULocationID=208, DOLocationID=242, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02875', pickup_datetime=datetime.datetime(2021, 2, 2, 15, 22, 24), dropoff_datetime=datetime.datetime(2021, 2, 2, 15, 33, 26), PULocationID=14, DOLocationID=14, SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_date

In [51]:
df_zones = spark.read.parquet('zones/')
df_zones.head(5)

[Row(LocationID='1', Borough='EWR', Zone='Newark Airport', service_zone='EWR'),
 Row(LocationID='2', Borough='Queens', Zone='Jamaica Bay', service_zone='Boro Zone'),
 Row(LocationID='3', Borough='Bronx', Zone='Allerton/Pelham Gardens', service_zone='Boro Zone'),
 Row(LocationID='4', Borough='Manhattan', Zone='Alphabet City', service_zone='Yellow Zone'),
 Row(LocationID='5', Borough='Staten Island', Zone='Arden Heights', service_zone='Boro Zone')]

In [54]:
df_tmp = df.join(df_zones, df.PULocationID == df_zones.LocationID, how='outer')
df_tmp = df_tmp.drop('LocationID')
df_tmp = df_tmp.withColumnRenamed('Borough', 'PU_Borough').withColumnRenamed('Zone', 'PU_Zone').withColumnRenamed('service_zone', 'PU_service_zone')
df_tmp.head(1)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02879', pickup_datetime=datetime.datetime(2021, 2, 2, 21, 42, 58), dropoff_datetime=datetime.datetime(2021, 2, 2, 22, 8, 4), PULocationID=31, DOLocationID=4, SR_Flag=None, PU_Borough='Bronx', PU_Zone='Bronx Park', PU_service_zone='Boro Zone')]

In [55]:
df_tmp = df_tmp.join(df_zones, df_tmp.DOLocationID == df_zones.LocationID, how='outer')
df_tmp = df_tmp.drop('LocationID')
df_with_zones = df_tmp.withColumnRenamed('Borough', 'DO_Borough').withColumnRenamed('Zone', 'DO_Zone').withColumnRenamed('service_zone', 'DO_service_zone')
df_with_zones.head(1)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 2, 17, 22, 10, 1), dropoff_datetime=datetime.datetime(2021, 2, 17, 22, 15, 17), PULocationID=31, DOLocationID=31, SR_Flag=None, PU_Borough='Bronx', PU_Zone='Bronx Park', PU_service_zone='Boro Zone', DO_Borough='Bronx', DO_Zone='Bronx Park', DO_service_zone='Boro Zone')]

In [61]:
# Question 6. Most common locations pair. Find the most common pickup-dropoff pair.
# For example: "Jamaica Bay / Clinton East"

# Enter two zone names separated by a slash

# If any of the zone names are unknown (missing), use "Unknown". For example, "Unknown / Clinton East".

from pyspark.sql.functions import coalesce

df_zone_concat = df_with_zones.withColumn("trip_zone",\
    concat(coalesce(df_with_zones.PU_Zone, lit('Unknown')), lit(' / '), coalesce(df_with_zones.DO_Zone, lit('Unknown'))))

df_zone_concat.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 2, 17, 22, 10, 1), dropoff_datetime=datetime.datetime(2021, 2, 17, 22, 15, 17), PULocationID=31, DOLocationID=31, SR_Flag=None, PU_Borough='Bronx', PU_Zone='Bronx Park', PU_service_zone='Boro Zone', DO_Borough='Bronx', DO_Zone='Bronx Park', DO_service_zone='Boro Zone', trip_zone='Bronx Park / Bronx Park'),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02395', pickup_datetime=datetime.datetime(2021, 2, 28, 15, 42, 59), dropoff_datetime=datetime.datetime(2021, 2, 28, 15, 49, 9), PULocationID=31, DOLocationID=31, SR_Flag=None, PU_Borough='Bronx', PU_Zone='Bronx Park', PU_service_zone='Boro Zone', DO_Borough='Bronx', DO_Zone='Bronx Park', DO_service_zone='Boro Zone', trip_zone='Bronx Park / Bronx Park'),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02864', pickup_datetime=datetime.datetime(2021, 2, 17, 14, 54, 50), dropoff_datetime=datetime.datetime(2021, 2, 17, 14, 

In [68]:
fhv_with_zones = df_zone_concat.registerTempTable('fhv_with_zones')

In [69]:
df_trips_common = spark.sql("""
SELECT 
    trip_zone,
    COUNT(1) AS number_trips
FROM
    fhv_with_zones
GROUP BY
    1
ORDER BY number_trips DESC  
""")

df_trips_common.head(5)

[Row(trip_zone='East New York / East New York', number_trips=45041),
 Row(trip_zone='Borough Park / Borough Park', number_trips=37329),
 Row(trip_zone='Canarsie / Canarsie', number_trips=28026),
 Row(trip_zone='Crown Heights North / Crown Heights North', number_trips=25976),
 Row(trip_zone='Bay Ridge / Bay Ridge', number_trips=17934)]